In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('report_edited.csv', encoding='latin1')


/var/folders/5b/zr381m555nndpqxs7__k9sv40000gq/T/ipykernel_733/2122878804.py:1: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('report_edited.csv', encoding='latin1')


In [15]:
excluded_terms = ['verizon', 'att', 'comcast', 'centurylink', 'telco']
def preprocess_text(text, excluded_terms):
    # Convert to lowercase, remove punctuation, etc. (existing preprocessing steps)

    # Exclude specific terms
    tokens = text.split()
    tokens = [word for word in tokens if word.lower() not in excluded_terms]
    
    # Continue with any further preprocessing like lemmatization, etc.
    return ' '.join(tokens)

# Apply preprocessing to your text data
df['processed_text'] = df['processed_text'].apply(lambda x: preprocess_text(x, excluded_terms))


In [16]:
from sklearn.decomposition import NMF
from sklearn.feature_extraction.text import TfidfVectorizer
# Remove rows where 'processed_text' is NaN
df = df.dropna(subset=['processed_text'])
# Use TF-IDF for vectorization
tfidf_vectorizer = TfidfVectorizer(max_features=10000)  # Adjust max_features
X = tfidf_vectorizer.fit_transform(df['processed_text'])

# Apply NMF
nmf = NMF(n_components=10, random_state=0)  # Adjust n_components (topics)
nmf.fit(X)

NMF(n_components=10, random_state=0)

In [5]:
def display_topics(model, feature_names, no_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print("Topic %d:" % (topic_idx))
        print(" ".join([feature_names[i] for i in topic.argsort()[:-no_top_words - 1:-1]]))



In [18]:
no_top_words = 25 # Number of top words to display for each topic
display_topics(nmf, tfidf_vectorizer.get_feature_names_out(), no_top_words)

Topic 0:
utc reported circuit outage transport signal beginning ntnoc system level cause center ffcc national loss problem operation issue network new oc power affected card cleared
Topic 1:
cell backhaul engineer interruption region notified service investigated discovered caused failed fiber damaged tower alarm monitored online commercial aware became detection came power back restore
Topic 2:
wireless site voip network cdma due cell wdfcc one macro service msc serving fiber outage incident caused operation isolated within cut reported center backhaul router
Topic 3:
path protect switched event working oc affected customer causing condition simplex card service ongoing issue multiplexer unknown failed failure outage cause time detected protected capable
Topic 4:
planned maintenance activity affecting upgrade service performing performed impacted engineer region metro ethernet interconnected network completed voip restoring may backhaul cell scheduled customer crq work
Topic 5:
telco 